In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
np.set_printoptions(precision=3)
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

1. Напишіть функцію гіпотези лінійної регресії у векторному вигляді

In [3]:
def linehypo(factX: np.ndarray | list,
             coefK: np.ndarray | list,
             indepB) -> np.ndarray:
    """
    0-hypothesis of linear regression:
      Y = factX *. coefK + indepB
    Example:
      print("Y =", linehypo(factX=[1,2], coefK=[3,4], indepB=5))
      Y = 16
      It means: Y = 1*3 + 2*4 + 5, 1*2 + 3*4 + 6 = 16
    Example 2:
      print("Y =", linehypo(factX=100, coefK=2, indepB=10))
      Y = 210
      It means: Y = 100 * 2 + 10 = 210
    Example 3:
      print("Y =", linehypo(factX=[[1,2],[11,22]], coefK=[3,4], indepB=5))
      Y = [ 16 126]
      It means: Y = [linehypo([1,2], [3,4], 5), linehypo([11,22], [3,4], 5)] = [16, 126]
    """
    factX = np.array(factX, dtype=np.float_)
    coefK = np.array(coefK, np.float_)
    Y = np.dot(factX, coefK) + indepB
    return Y
print("Y =", linehypo(factX=100, coefK=2, indepB=10))
print("Y =", linehypo(factX=[[100],[200],[300]], coefK=[2], indepB=10))
print("Y =", linehypo(factX=[1,2], coefK=[3,4], indepB=5))
print("Y =", linehypo(factX=np.array([[1,2],[11,22],[111,222]]), coefK=[3,4], indepB=5))

Y = 210.0
Y = [210. 410. 610.]
Y = 16.0
Y = [  16.  126. 1226.]


2. Створіть функцію для обчислення функції втрат у векторному вигляді

In [4]:
def linehypoloss(realY: np.array, # results from table
                 factX: np.array, # array of factor vectors from table
                 coefK: np.array, # calc vector (or single value) of coef
                 indepB = 0       # calc vector (or single value) of independed values
                 ):
    """
    Linear Hypothesis Loss Function.

    print(linehypoloss(realY=[6.1, 6.8, 8.05], factX=[1, 2, 3], coefK=1, indepB=5))
    0.008750000000000013
    (0.1**2 + (-0.2)**2 + 0.05**2) / (3 * 2) = 0.00875

    print(linehypoloss(realY=[6.1, 6.8, 8.05], factX=np.array([[1,1], [2,2], [3,3]]), coefK=[1,2], indepB=5))
    ((6.1-(1*1+1*2+5))**2 + (6.8-(2*1+2*2+5))**2 + (8.05-(3*1+3*2+5))**2) / (3 << 1) = 9.442083(3)
    """
    realY = np.array(realY, dtype=np.float_)
    factX = np.array(factX, dtype=np.float_)
    coefK = np.array(coefK, dtype=np.float_)
    if factX.shape[0] != realY.shape[0]:
        raise ValueError(f"In linehipoloss() amount of elements = {realY.shape[0]} in 'realY' differs from amount of rows = {factX.shape[0]} in 'factX'")
    loss = ((realY - linehypo(factX, coefK, indepB)) ** 2).sum() / (factX.shape[0] << 1)
    return loss
print("Calc:", linehypoloss(realY=[6.1, 6.8, 8.05], factX=np.array([1, 2, 3]), coefK=1, indepB=5))
print("Test:", (0.1**2 + (-0.2)**2 + 0.05**2) / (3 << 1))
print("Calc:", linehypoloss(realY=[6.1, 6.8, 8.05], factX=np.array([[1,1], [2,2], [3,3]]), coefK=[1,2], indepB=5))
print("Test:", ((6.1-(1*1+1*2+5))**2 + (6.8-(2*1+2*2+5))**2 + (8.05-(3*1+3*2+5))**2) / (3 << 1))

Calc: 0.008750000000000013
Test: 0.008750000000000003
Calc: 9.442083333333331
Test: 9.442083333333331


3. Реалізуйте один крок градієнтного спуску

In [5]:
def descent_step(step, realY, factX, coefK, indepB):
    """
    One Step to Calc of Fradient Descention
    """
    realY = np.array(realY, dtype=np.float_)
    factX = np.array(factX, dtype=np.float_)
    coefK = np.array(coefK, dtype=np.float_)
    nfacts = factX.shape[0]
    if nfacts != realY.shape[0]:
        raise ValueError(f"In descent_step() amount of elements = {realY.shape[0]} in 'realY' differs from amount of rows = {nfacts} in 'factX'")
    multor = step / nfacts
    vHsY = linehypo(factX, coefK, indepB) - realY
    indepB -= vHsY.sum() * multor
    # print("[~x~]", "vHsY:", vHsY, "indepB =", indepB)
    vHsY = linehypo(factX, coefK, indepB) - realY
    if coefK.shape:
        for i in range(coefK.shape[0]):
            # print("vHsY =", vHsY, "*", factX[:,i], "=", vHsY * factX[:,i])
            coefK[i] -= (vHsY * factX[:,i]).sum() * multor
    else:
        vHsY = linehypo(factX, coefK, indepB) - realY
        coefK -= (vHsY * factX).sum() * multor
    return coefK, indepB

step=0.005; realY=[8, 11, 14, 17, 21]; factX=[[1,1], [2,2], [3,3], [10,1], [10,3]]; coefK=[0,0]; indepB=0
THRESHOLD = 0.000000000000000001
for _ in range(5000000):
    if linehypoloss(realY=realY, factX=factX, coefK=coefK, indepB=indepB) < THRESHOLD:
      break
    coefK, indepB = descent_step(step=step, realY=realY, factX=factX, coefK=coefK, indepB=indepB);
print("Test:", "\n" "coefK =", coefK, "\n" "indepB =", indepB, "\n" "realY", linehypo(factX, coefK, indepB))

Test: 
coefK = [1. 2.] 
indepB = 4.999999996235656 
realY [ 8. 11. 14. 17. 21.]


4. Знайдіть найкращі параметри w⃗ для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень

In [6]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data-science-hw03/Housing.csv')
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [7]:
realY = np.array(df['price'].tolist(), dtype=np.float_)
factX = np.array(df[['area', 'bedrooms', 'bathrooms']].values.tolist(), dtype=np.float_)
realY[:5], factX[:5]

(array([13300000., 12250000., 12250000., 12215000., 11410000.]),
 array([[7.42e+03, 4.00e+00, 2.00e+00],
        [8.96e+03, 4.00e+00, 4.00e+00],
        [9.96e+03, 3.00e+00, 2.00e+00],
        [7.50e+03, 4.00e+00, 2.00e+00],
        [7.42e+03, 4.00e+00, 1.00e+00]]))

In [8]:
def normalist(ls: np.array) -> np.array:
    """
    Data normalisation
    """
    ls = np.array(ls, np.float_)
    if len(ls.shape) == 1:
        lsavg = np.mean(ls)
        lssz = np.max(ls)
        lssz -= np.min(ls)
        return (ls - lsavg) / lssz, np.array([lssz, lsavg], dtype=np.float_)
    else:
        lsret = []
        for i in range(ls.shape[1]):
            li = ls[:,i]
            lsavg = np.mean(li)
            lssz = np.max(li)
            lssz -= np.min(li)
            ls[:,i] = (li - lsavg) / lssz
            lsret.append((lssz, lsavg))
        return ls, np.array(lsret, dtype=np.float_)
realY, realY_r = normalist(realY)
factX, factX_r = normalist(factX)
realY[:5], realY_r, factX[:5], factX_r

(array([0.739, 0.648, 0.648, 0.645, 0.575]),
 array([11550000.   ,  4766729.248]),
 array([[ 0.156,  0.207,  0.238],
        [ 0.262,  0.207,  0.905],
        [ 0.331,  0.007,  0.238],
        [ 0.161,  0.207,  0.238],
        [ 0.156,  0.207, -0.095]]),
 array([[1.455e+04, 5.151e+03],
        [5.000e+00, 2.965e+00],
        [3.000e+00, 1.286e+00]]))

In [9]:
def denormalist(ls: np.array, lsret: list) -> np.array:
    ls = np.array(ls)
    if len(ls.shape) == 1:
        lssz, lsavg = lsret
        return ls * lssz + lsavg
    else:
        for i in range(ls.shape[1]):
            li = ls[:,i]
            lssz, lsavg = lsret[i]
            ls[:,i] = li * lssz + lsavg
        return ls
denormalist(factX, factX_r)[:5]

array([[7.42e+03, 4.00e+00, 2.00e+00],
       [8.96e+03, 4.00e+00, 4.00e+00],
       [9.96e+03, 3.00e+00, 2.00e+00],
       [7.50e+03, 4.00e+00, 2.00e+00],
       [7.42e+03, 4.00e+00, 1.00e+00]])

**Making Gradient Descention with Real Data**

---


In [10]:
step=1; coefK=[0, 0, 0]; indepB=0
THRESHOLD = 0.001
for at in range(500001):
    if linehypoloss(realY=realY, factX=factX, coefK=coefK, indepB=indepB) < THRESHOLD:
      break
    coefK, indepB = descent_step(step=step, realY=realY, factX=factX, coefK=coefK, indepB=indepB);
print("Steps:", at, "\n" "coefK =", coefK, "\n" "indepB =", indepB,
      "\n" "Loss =", np.mean(linehypoloss(realY=realY, factX=factX, coefK=coefK, indepB=indepB)))

Steps: 500000 
coefK = [0.477 0.176 0.36 ] 
indepB = 4.889055521285093e-17 
Loss = 0.006713405108514905


5. Знайдіть ці ж параметри за допомогою аналітичного рішення

**Analytical Solution**

---



In [11]:
coefK_a = np.dot(np.dot(np.linalg.inv(np.dot(factX.T, factX)), factX.T), realY)
coefK_a

array([0.477, 0.176, 0.36 ])

6. Порівняйте отримані результати

Є повне співпадіння:
[0.477 0.176 0.36 ] = [0.477, 0.176, 0.36 ]